## Uavs Collision

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [1]:
import os
import subprocess
from pymavlink import mavutil

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns
from helpers.gazebo_world import update_world
from helpers.math import manhattan_distance
from helpers.navegation_logic import find_path,find_best_waypoint

## Kill all related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

## Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/attack_city.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

# Choose Simulator

In [4]:
simulator ='gazebo' # 'QGroundControl' #

# Choose Initial Position

In [5]:
offsets = [(40, -10, 0, 0),(15, -10, 0, 0)]#
n_uavs = len(offsets)

                # lat,long,alt,heading
if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,90)
    spawns=find_spawns(home_position, offsets)

## Waypoint seletion algorithm

In [6]:
## right now wps positions are relative to uav offset
wps_2d=[(0, 0),
        (0, -30),
        (-25,0),
        (25,0),
        (-25, -30),
        (-40, 0),
        (-40, -30),
        (-40, -14)]

start_pos = [wps_2d[0],wps_2d[0]]
target_pos = [wps_2d[-1],wps_2d[3]]

# Define start, target, and waypoints

all_waypoints = wps_2d[1:]

# Compute path
for i in range(n_uavs):
        print(f'uav-{i} Best path: {find_path(start_pos[i], target_pos[i], all_waypoints.copy())}')    


uav-0 Best path: [(0, 0), (-25, 0), (-40, 0), (-40, -14)]
uav-1 Best path: [(0, 0), (25, 0)]


In [7]:
wps_2d[3]

(25, 0)

## Launch Simulator

In [8]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(f"drone{i+1}", east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

## Create Plan

In [9]:
plan=['check_prearm','check_pos_est','mode_stabilize','mode_guided','arm','takeoff','fly','land']
uavs_arg = []
altitude = 5
path_2d= [(0, 0),(-20, -10)]#      
path = [(x, y, -altitude) for x, y in path_2d]
for i in range(n_uavs):
    uav_arg={'waypoints':path,
             'plan': plan,
             'wp_margin': 0.5}
    uavs_arg.append(uav_arg)

In [10]:
all_waypoints 

[(0, -30), (-25, 0), (25, 0), (-25, -30), (-40, 0), (-40, -30), (-40, -14)]

# Launch Vehicle

In [11]:
vehicle_path = os.path.expanduser(ardupilot_vehicle_path)

In [12]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

## Create UavLogic

In [13]:
uavs=[]
for i,uav_arg in enumerate(uavs_arg):
    connection = mavutil.mavlink_connection(f'udp:127.0.0.1:{14551+10*i}')
    connection.wait_heartbeat()
    uav=VehicleLogic(connection,**uav_arg)
    uavs.append(uav)

vehicle 1 created
vehicle 2 created


## Execute Plan

In [ ]:
mission_incomplete = n_uavs*[True]
target_reached =n_uavs*[False] 
waypoints = [all_waypoints.copy() for _ in range(n_uavs)]
wp=n_uavs*[0]
while any(mission_incomplete):
    for i,uav in enumerate(uavs):
        mission_incomplete[i] = uav.act_plan()
        if uav.is_reached and (not target_reached[i]):
            wp[i]+=1
            target=target_pos[i]
            current = uav.get_position()[:2]
            # GPS spoofing
            if manhattan_distance(current,(-25, 0))<1:
                current=(-25, -20)
            best_waypoint = find_best_waypoint(current, target, waypoints[i])
            print(waypoints[i])
            print(best_waypoint)
            waypoints[i].remove(best_waypoint) 
            path_2d=[current,best_waypoint]
            if best_waypoint!=target:
                path_2d.append(target)
            else:
                target_reached[i]=True
            print(f'--- waypoint {wp[i]} reached --- ')
            print(f'current position: ({path_2d[0][0]:.2f},{path_2d[0][1]:.2f})')
            print(f'next waypoint: {path_2d[1]}')
            print(f'target position: {path_2d[-1]}')
            path = [(x, y, -altitude) for x, y in path_2d]
            uav.update_waypoints(path)
        

vehicle 1: action land is done
vehicle 2: action land is done
vehicle 1: action check_prearm is done
vehicle 2: action check_prearm is done


## Kill all related process

In [ ]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")